<a href="https://colab.research.google.com/github/Forbusinessuseyukikoishiguro/---PDF-20250403/blob/main/%E3%82%AB%E3%82%B9%E3%82%BF%E3%83%9E%E3%82%A4%E3%82%BA%E5%8F%AF%E8%83%BD%E3%81%AANCBI%E8%AB%96%E6%96%87%E6%A4%9C%E7%B4%A2%E3%83%BB%E4%BF%9D%E5%AD%98%E3%83%84%E3%83%BC%E3%83%AB%EF%BC%88Google_Colab%E7%94%A8%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#カスタマイズ可能なNCBI論文検索・保存ツール（Google Colab用）
主な機能

詳細な検索条件：

キーワード、著者名、ジャーナル名、出版年などで検索
最大200件までの論文を一度に取得可能


高度な絞り込み機能：

検索結果を日付（年・月）やジャーナルでさらに絞り込み
著者、キーワード、言語、論文種類での絞り込み
フルテキスト有のみ表示するオプション


柔軟な保存先指定：

任意のファイル名とパスを指定可能
Google Driveの任意フォルダに直接保存可能
検索結果と絞り込み結果を別々に保存可能


データ分析・可視化：

出版年別の論文数グラフ表示
ジャーナル別の統計情報と円グラフ表示



使い方

検索タブ：基本的な検索条件を設定して初期検索を実行
絞り込みタブ：検索結果をさらに細かい条件で絞り込み
エクスポートタブ：

ファイル名と保存先パスを指定（例：/content/drive/MyDrive/研究/論文データ/）
「現在の検索結果をエクスポート」または「絞り込み後の結果をエクスポート」ボタンをクリック


可視化タブ：検索結果のグラフ表示と統計分析

In [1]:
# 高度にカスタマイズ可能なNCBI論文検索ツール
# Google Colab用・詳細な検索条件指定＆保存先カスタマイズ機能付き

# 必要なライブラリのインストール（最初の実行時のみ）
# !pip install pandas requests ipywidgets matplotlib seaborn tqdm

import requests
import xml.etree.ElementTree as ET
import pandas as pd
import time
import os
import datetime
from IPython.display import display, HTML, clear_output
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import ipywidgets as widgets

class AdvancedNCBISearch:
    """
    NCBIの論文を検索・取得し、カスタマイズした条件で絞り込み可能なクラス
    """

    def __init__(self):
        self.base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
        self.current_articles = []  # 現在の検索結果
        self.filtered_articles = []  # フィルタリング後の結果

    def search_pubmed(self, query_params):
        """
        PubMedで検索を実行

        Parameters:
        query_params (dict): 検索パラメータ

        Returns:
        list: 論文情報のリスト
        """
        # 検索クエリの構築
        query_parts = []

        # キーワード
        if query_params["keywords"]:
            query_parts.append(f"({query_params['keywords']})")

        # 著者
        if query_params["author"]:
            query_parts.append(f"{query_params['author']}[Author]")

        # ジャーナル
        if query_params["journal"]:
            query_parts.append(f"{query_params['journal']}[Journal]")

        # 出版年
        if query_params["year_from"] and query_params["year_to"]:
            query_parts.append(f"{query_params['year_from']}:{query_params['year_to']}[pdat]")
        elif query_params["year_from"]:
            query_parts.append(f"{query_params['year_from']}[pdat]")
        elif query_params["year_to"]:
            query_parts.append(f":{query_params['year_to']}[pdat]")

        # 最終クエリ
        final_query = " AND ".join(query_parts)

        # 検索実行
        search_url = f"{self.base_url}esearch.fcgi"
        search_params = {
            "db": "pubmed",
            "term": final_query,
            "retmax": query_params["max_results"],
            "usehistory": "y",
            "retmode": "xml"
        }

        try:
            print(f"検索クエリ: {final_query}")
            search_response = requests.get(search_url, params=search_params)
            search_response.raise_for_status()

            search_root = ET.fromstring(search_response.content)

            # 検索結果カウント
            count_elem = search_root.find(".//Count")
            if count_elem is None or int(count_elem.text) == 0:
                print("検索結果が見つかりませんでした。検索条件を変更してみてください。")
                return []

            # WebEnvとQueryKey取得
            web_env = search_root.find(".//WebEnv").text
            query_key = search_root.find(".//QueryKey").text
            count = int(count_elem.text)

            print(f"検索結果: {count}件 (最大{query_params['max_results']}件を表示)")

            # 論文詳細を取得
            fetch_url = f"{self.base_url}efetch.fcgi"
            fetch_params = {
                "db": "pubmed",
                "query_key": query_key,
                "WebEnv": web_env,
                "retmax": query_params["max_results"],
                "retmode": "xml",
                "rettype": "abstract"
            }

            fetch_response = requests.get(fetch_url, params=fetch_params)
            fetch_response.raise_for_status()

            fetch_root = ET.fromstring(fetch_response.content)

            # 論文データを抽出
            articles = []

            for article_elem in fetch_root.findall(".//PubmedArticle"):
                # PMID
                pmid_elem = article_elem.find(".//PMID")
                pmid = pmid_elem.text if pmid_elem is not None else "Unknown"

                # タイトル
                title_elem = article_elem.find(".//ArticleTitle")
                title = title_elem.text if title_elem is not None else "No title available"

                # 著者
                authors = []
                author_list = article_elem.findall(".//Author")

                for author in author_list:
                    last_name = author.find("LastName")
                    fore_name = author.find("ForeName")

                    if last_name is not None and fore_name is not None:
                        authors.append(f"{last_name.text} {fore_name.text}")
                    elif last_name is not None:
                        authors.append(last_name.text)

                # ジャーナル
                journal_elem = article_elem.find(".//Journal/Title")
                journal = journal_elem.text if journal_elem is not None else "Unknown journal"

                # ISSN
                issn_elem = article_elem.find(".//ISSN")
                issn = issn_elem.text if issn_elem is not None else "Unknown"

                # 出版年月日
                year = "Unknown"
                month = "Unknown"
                day = "Unknown"

                year_elem = article_elem.find(".//PubDate/Year")
                if year_elem is not None:
                    year = year_elem.text

                month_elem = article_elem.find(".//PubDate/Month")
                if month_elem is not None:
                    month = month_elem.text

                day_elem = article_elem.find(".//PubDate/Day")
                if day_elem is not None:
                    day = day_elem.text

                # MedlineDate形式の場合
                if year == "Unknown":
                    medline_date = article_elem.find(".//PubDate/MedlineDate")
                    if medline_date is not None and medline_date.text:
                        # 年を抽出
                        import re
                        year_match = re.search(r'(\d{4})', medline_date.text)
                        if year_match:
                            year = year_match.group(1)

                        # 月を抽出
                        month_match = re.search(r'(\w{3})', medline_date.text)
                        if month_match:
                            month = month_match.group(1)

                # DOI
                doi = None
                for id_elem in article_elem.findall(".//ArticleId"):
                    if id_elem.get("IdType") == "doi":
                        doi = id_elem.text
                        break

                # PubMed URL
                pubmed_url = f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/"

                # PMC ID とURL
                pmc_id = None
                pmc_url = None
                for id_elem in article_elem.findall(".//ArticleId"):
                    if id_elem.get("IdType") == "pmc":
                        pmc_id = id_elem.text
                        pmc_url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/{pmc_id}/"
                        break

                # 抄録
                abstract_text = ""
                abstract_elems = article_elem.findall(".//AbstractText")
                for abstract_elem in abstract_elems:
                    if abstract_elem.text:
                        label = abstract_elem.get("Label")
                        if label:
                            abstract_text += f"{label}: "
                        abstract_text += abstract_elem.text + " "

                if not abstract_text:
                    abstract_text = "No abstract available"

                # キーワード
                keywords = []
                keyword_elems = article_elem.findall(".//Keyword")
                for keyword_elem in keyword_elems:
                    if keyword_elem.text:
                        keywords.append(keyword_elem.text)

                # 言語
                language_elem = article_elem.find(".//Language")
                language = language_elem.text if language_elem is not None else "Unknown"

                # 論文種類
                publication_types = []
                pub_type_elems = article_elem.findall(".//PublicationType")
                for pub_type_elem in pub_type_elems:
                    if pub_type_elem.text:
                        publication_types.append(pub_type_elem.text)

                # 論文情報を辞書に格納
                article_info = {
                    "pmid": pmid,
                    "title": title,
                    "authors": authors,
                    "journal": journal,
                    "issn": issn,
                    "year": year,
                    "month": month,
                    "day": day,
                    "doi": doi,
                    "pubmed_url": pubmed_url,
                    "pmc_id": pmc_id,
                    "pmc_url": pmc_url,
                    "abstract": abstract_text,
                    "keywords": keywords,
                    "language": language,
                    "publication_types": publication_types
                }

                articles.append(article_info)

            self.current_articles = articles
            self.filtered_articles = articles.copy()  # 初期状態では全ての記事
            return articles

        except requests.exceptions.RequestException as e:
            print(f"APIリクエストエラー: {e}")
            return []
        except Exception as e:
            print(f"エラーが発生しました: {e}")
            return []

    def filter_articles(self, filter_params):
        """
        検索結果をさらに条件で絞り込む

        Parameters:
        filter_params (dict): フィルタリングパラメータ

        Returns:
        list: フィルタリングされた論文情報のリスト
        """
        if not self.current_articles:
            print("フィルタリングするデータがありません。先に検索を実行してください。")
            return []

        filtered = self.current_articles.copy()

        # ジャーナル名でフィルタリング
        if filter_params.get("journal"):
            journal_filter = filter_params["journal"].lower()
            filtered = [a for a in filtered if journal_filter in a["journal"].lower()]

        # 年でフィルタリング
        if filter_params.get("year_from"):
            try:
                year_from = int(filter_params["year_from"])
                filtered = [a for a in filtered if a["year"] != "Unknown" and int(a["year"]) >= year_from]
            except ValueError:
                pass

        if filter_params.get("year_to"):
            try:
                year_to = int(filter_params["year_to"])
                filtered = [a for a in filtered if a["year"] != "Unknown" and int(a["year"]) <= year_to]
            except ValueError:
                pass

        # 月でフィルタリング
        if filter_params.get("month"):
            month_filter = filter_params["month"].lower()
            # 月の文字列表現を数値に変換するための辞書
            month_map = {
                "jan": "1", "january": "1",
                "feb": "2", "february": "2",
                "mar": "3", "march": "3",
                "apr": "4", "april": "4",
                "may": "5",
                "jun": "6", "june": "6",
                "jul": "7", "july": "7",
                "aug": "8", "august": "8",
                "sep": "9", "september": "9",
                "oct": "10", "october": "10",
                "nov": "11", "november": "11",
                "dec": "12", "december": "12"
            }

            # 月名の場合は数値に変換
            if month_filter in month_map:
                month_filter = month_map[month_filter]

            filtered = [a for a in filtered if a["month"] != "Unknown" and
                       (a["month"].lower() == month_filter or
                        (a["month"] in month_map and month_map[a["month"].lower()] == month_filter) or
                        a["month"] == month_filter)]

        # 著者でフィルタリング
        if filter_params.get("author"):
            author_filter = filter_params["author"].lower()
            filtered = [a for a in filtered if any(author_filter in author.lower() for author in a["authors"])]

        # キーワードでフィルタリング
        if filter_params.get("keyword"):
            keyword_filter = filter_params["keyword"].lower()
            filtered = [a for a in filtered if (
                keyword_filter in a["title"].lower() or
                keyword_filter in a["abstract"].lower() or
                any(keyword_filter in kw.lower() for kw in a["keywords"])
            )]

        # 言語でフィルタリング
        if filter_params.get("language"):
            language_filter = filter_params["language"].lower()
            filtered = [a for a in filtered if language_filter in a["language"].lower()]

        # 論文種類でフィルタリング
        if filter_params.get("publication_type"):
            pub_type_filter = filter_params["publication_type"].lower()
            filtered = [a for a in filtered if any(pub_type_filter in pt.lower() for pt in a["publication_types"])]

        # フルテキスト有無でフィルタリング
        if filter_params.get("has_fulltext") and filter_params["has_fulltext"]:
            filtered = [a for a in filtered if a["pmc_id"] is not None]

        self.filtered_articles = filtered
        return filtered

    def export_to_csv(self, articles, filepath=None):
        """
        検索結果をCSVファイルに保存

        Parameters:
        articles (list): 論文情報のリスト
        filepath (str): 保存先のファイルパス（Noneの場合はデフォルト名で保存）

        Returns:
        str: 保存したファイルパス
        """
        if not articles:
            print("エクスポートするデータがありません。")
            return None

        # デフォルトのファイル名生成
        if filepath is None:
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            filepath = f"pubmed_results_{timestamp}.csv"

        # CSVデータの作成
        data = []
        for article in articles:
            row = {
                "PMID": article["pmid"],
                "Title": article["title"],
                "Authors": ", ".join(article["authors"]) if article["authors"] else "",
                "Journal": article["journal"],
                "ISSN": article["issn"],
                "Year": article["year"],
                "Month": article["month"],
                "Day": article["day"],
                "DOI": article["doi"] if article["doi"] else "",
                "PubMed URL": article["pubmed_url"],
                "PMC ID": article["pmc_id"] if article["pmc_id"] else "",
                "PMC URL": article["pmc_url"] if article["pmc_url"] else "",
                "Keywords": ", ".join(article["keywords"]) if article["keywords"] else "",
                "Language": article["language"],
                "Publication Types": ", ".join(article["publication_types"]) if article["publication_types"] else "",
                "Abstract": article["abstract"]
            }
            data.append(row)

        df = pd.DataFrame(data)

        # ディレクトリが存在しない場合は作成
        directory = os.path.dirname(filepath)
        if directory and not os.path.exists(directory):
            os.makedirs(directory)

        df.to_csv(filepath, index=False, encoding="utf-8-sig")
        print(f"データを {filepath} に保存しました")

        # Google Colabでダウンロードリンクを表示
        try:
            from google.colab import files
            files.download(filepath)
        except:
            pass

        return filepath

    def get_journal_statistics(self, articles):
        """
        ジャーナル別の統計情報を取得

        Parameters:
        articles (list): 論文情報のリスト

        Returns:
        pandas.DataFrame: ジャーナル別統計情報
        """
        if not articles:
            return None

        journal_counts = {}

        for article in articles:
            journal = article["journal"]
            if journal in journal_counts:
                journal_counts[journal] += 1
            else:
                journal_counts[journal] = 1

        # DataFrameに変換
        df = pd.DataFrame({
            "ジャーナル名": list(journal_counts.keys()),
            "論文数": list(journal_counts.values())
        })

        # 論文数の多い順にソート
        df = df.sort_values("論文数", ascending=False).reset_index(drop=True)

        return df

    def plot_year_distribution(self, articles):
        """
        出版年の分布をプロット

        Parameters:
        articles (list): 論文情報のリスト
        """
        if not articles:
            print("表示するデータがありません。")
            return

        # 年を抽出
        years = []
        for article in articles:
            if article["year"] and article["year"] != "Unknown":
                try:
                    year = int(article["year"])
                    years.append(year)
                except ValueError:
                    continue

        if not years:
            print("年データがありません。")
            return

        # 年ごとの論文数をカウント
        year_counts = pd.Series(years).value_counts().sort_index()

        # グラフ作成
        plt.figure(figsize=(12, 6))
        bars = sns.barplot(x=year_counts.index, y=year_counts.values)

        # 棒グラフに値を表示
        for i, v in enumerate(year_counts.values):
            bars.text(i, v + 0.1, str(v), ha='center')

        plt.title("出版年別論文数")
        plt.xlabel("出版年")
        plt.ylabel("論文数")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

# インタラクティブ検索UIの実装
def create_advanced_search_ui():
    """
    高度な検索・フィルタリング・エクスポート機能を持つ検索インターフェースを作成
    """
    # NCBIサーチャーのインスタンス化
    searcher = AdvancedNCBISearch()

    # タブの作成
    tabs = widgets.Tab()
    search_tab = widgets.VBox()
    filter_tab = widgets.VBox()
    export_tab = widgets.VBox()
    visualize_tab = widgets.VBox()

    tabs.children = [search_tab, filter_tab, export_tab, visualize_tab]
    tabs.set_title(0, "検索")
    tabs.set_title(1, "絞り込み")
    tabs.set_title(2, "エクスポート")
    tabs.set_title(3, "可視化")

    # 検索タブの要素
    search_title = widgets.HTML("<h3>論文検索条件</h3>")

    keyword_input = widgets.Text(
        value='',
        placeholder='キーワード（例: cancer therapy）',
        description='キーワード:',
        layout=widgets.Layout(width='100%')
    )

    author_input = widgets.Text(
        value='',
        placeholder='例: Smith J',
        description='著者名:',
        layout=widgets.Layout(width='100%')
    )

    journal_input = widgets.Text(
        value='',
        placeholder='例: Nature',
        description='ジャーナル:',
        layout=widgets.Layout(width='100%')
    )

    year_from = widgets.Text(
        value='',
        placeholder='例: 2020',
        description='出版年（開始）:',
        layout=widgets.Layout(width='48%')
    )

    year_to = widgets.Text(
        value='',
        placeholder='例: 2023',
        description='出版年（終了）:',
        layout=widgets.Layout(width='48%')
    )

    max_results = widgets.IntSlider(
        value=50,
        min=10,
        max=200,
        step=10,
        description='最大取得件数:',
        layout=widgets.Layout(width='100%')
    )

    search_button = widgets.Button(
        description='検索実行',
        button_style='primary',
        icon='search',
        layout=widgets.Layout(width='100%')
    )

    search_output = widgets.Output()

    # 検索タブのレイアウト
    search_tab.children = [
        search_title,
        keyword_input,
        author_input,
        journal_input,
        widgets.HBox([year_from, year_to]),
        max_results,
        search_button,
        search_output
    ]

    # 絞り込みタブの要素
    filter_title = widgets.HTML("<h3>検索結果の絞り込み</h3>")
    filter_journal = widgets.Text(
        value='',
        placeholder='例: Nature',
        description='ジャーナル名:',
        layout=widgets.Layout(width='100%')
    )

    filter_year_from = widgets.Text(
        value='',
        placeholder='例: 2021',
        description='出版年（開始）:',
        layout=widgets.Layout(width='48%')
    )

    filter_year_to = widgets.Text(
        value='',
        placeholder='例: 2022',
        description='出版年（終了）:',
        layout=widgets.Layout(width='48%')
    )

    filter_month = widgets.Text(
        value='',
        placeholder='例: Jan または 1',
        description='出版月:',
        layout=widgets.Layout(width='100%')
    )

    filter_author = widgets.Text(
        value='',
        placeholder='例: Smith',
        description='著者名:',
        layout=widgets.Layout(width='100%')
    )

    filter_keyword = widgets.Text(
        value='',
        placeholder='例: therapy',
        description='キーワード:',
        layout=widgets.Layout(width='100%')
    )

    filter_language = widgets.Text(
        value='',
        placeholder='例: eng',
        description='言語:',
        layout=widgets.Layout(width='100%')
    )

    filter_publication_type = widgets.Text(
        value='',
        placeholder='例: Review',
        description='論文種類:',
        layout=widgets.Layout(width='100%')
    )

    filter_has_fulltext = widgets.Checkbox(
        value=False,
        description='フルテキスト有のみ表示',
        layout=widgets.Layout(width='100%')
    )

    filter_button = widgets.Button(
        description='絞り込み実行',
        button_style='success',
        icon='filter',
        layout=widgets.Layout(width='100%')
    )

    filter_output = widgets.Output()

    # 絞り込みタブのレイアウト
    filter_tab.children = [
        filter_title,
        widgets.HTML("<p>以下の条件で検索結果をさらに絞り込みます</p>"),
        filter_journal,
        widgets.HBox([filter_year_from, filter_year_to]),
        filter_month,
        filter_author,
        filter_keyword,
        filter_language,
        filter_publication_type,
        filter_has_fulltext,
        filter_button,
        filter_output
    ]

    # エクスポートタブの要素
    export_title = widgets.HTML("<h3>検索結果のエクスポート</h3>")

    export_filename = widgets.Text(
        value='',
        placeholder='例: pubmed_results.csv',
        description='ファイル名:',
        layout=widgets.Layout(width='100%')
    )

    export_path = widgets.Text(
        value='',
        placeholder='例: /content/drive/MyDrive/研究/論文データ/ (空欄の場合はデフォルト)',
        description='保存先:',
        layout=widgets.Layout(width='100%')
    )

    export_current_button = widgets.Button(
        description='現在の検索結果をエクスポート',
        button_style='info',
        icon='download',
        layout=widgets.Layout(width='100%')
    )

    export_filtered_button = widgets.Button(
        description='絞り込み後の結果をエクスポート',
        button_style='info',
        icon='download',
        layout=widgets.Layout(width='100%')
    )

    export_output = widgets.Output()

    # エクスポートタブのレイアウト
    export_tab.children = [
        export_title,
        widgets.HTML("<p>検索結果をCSVファイルとして保存します</p>"),
        export_filename,
        export_path,
        export_current_button,
        export_filtered_button,
        export_output
    ]

    # 可視化タブの要素
    visualize_title = widgets.HTML("<h3>検索結果の可視化</h3>")

    visualize_year_button = widgets.Button(
        description='出版年別グラフ表示',
        button_style='warning',
        icon='bar-chart',
        layout=widgets.Layout(width='100%')
    )

    visualize_journal_button = widgets.Button(
        description='ジャーナル別統計表示',
        button_style='warning',
        icon='pie-chart',
        layout=widgets.Layout(width='100%')
    )

    visualize_output = widgets.Output()

    # 可視化タブのレイアウト
    visualize_tab.children = [
        visualize_title,
        widgets.HTML("<p>検索結果を視覚的に分析します</p>"),
        visualize_year_button,
        visualize_journal_button,
        visualize_output
    ]

    # 検索ボタンのクリックイベント
    def on_search_button_clicked(b):
        with search_output:
            clear_output()

            # 検索パラメータの取得
            params = {
                "keywords": keyword_input.value.strip(),
                "author": author_input.value.strip(),
                "journal": journal_input.value.strip(),
                "year_from": year_from.value.strip(),
                "year_to": year_to.value.strip(),
                "max_results": max_results.value
            }

            # 少なくとも1つのパラメータが必要
            if not any([params["keywords"], params["author"], params["journal"], params["year_from"], params["year_to"]]):
                print("少なくとも1つの検索条件を入力してください。")
                return

            print("検索中...")
            articles = searcher.search_pubmed(params)

            if not articles:
                return

            # 検索結果の表示
            print(f"\n検索結果: {len(articles)}件")

            # DataFrameの作成
            df = pd.DataFrame([
                {
                    "PMID": article["pmid"],
                    "タイトル": article["title"][:50] + "..." if len(article["title"]) > 50 else article["title"],
                    "著者": (", ".join(article["authors"][:2]) + "..." if len(article["authors"]) > 2 else ", ".join(article["authors"])),
                    "ジャーナル": article["journal"],
                    "年": article["year"],
                    "URL": f"<a href='{article['pubmed_url']}' target='_blank'>PubMed</a>"
                }
                for article in articles
            ])

            # DataFrameを表示
            display(HTML(df.to_html(escape=False, index=False)))

            # 検索結果があれば次のタブへのナビゲーションを表示
            if articles:
                display(HTML("<p><b>次のステップ:</b> 「絞り込み」タブや「エクスポート」タブを使って検索結果をさらに活用できます</p>"))

    # 絞り込みボタンのクリックイベント
    def on_filter_button_clicked(b):
        with filter_output:
            clear_output()

            if not searcher.current_articles:
                print("絞り込む検索結果がありません。先に検索を実行してください。")
                return

            # フィルタパラメータの取得
            filter_params = {
                "journal": filter_journal.value.strip(),
                "year_from": filter_year_from.value.strip(),
                "year_to": filter_year_to.value.strip(),
                "month": filter_month.value.strip(),
                "author": filter_author.value.strip(),
                "keyword": filter_keyword.value.strip(),
                "language": filter_language.value.strip(),
                "publication_type": filter_publication_type.value.strip(),
                "has_fulltext": filter_has_fulltext.value
            }

            # フィルタリング実行
            filtered_articles = searcher.filter_articles(filter_params)

            if not filtered_articles:
                print("条件に一致する論文がありませんでした。絞り込み条件を緩めてください。")
                return

            # 結果表示
            print(f"絞り込み結果: {len(filtered_articles)}件 / {len(searcher.current_articles)}件中")

            # DataFrameの作成
            df = pd.DataFrame([
                {
                    "PMID": article["pmid"],
                    "タイトル": article["title"][:50] + "..." if len(article["title"]) > 50 else article["title"],
                    "ジャーナル": article["journal"],
                    "年月": f"{article['year']}/{article['month']}",
                    "著者": (", ".join(article["authors"][:2]) + "..." if len(article["authors"]) > 2 else ", ".join(article["authors"])),
                    "URL": f"<a href='{article['pubmed_url']}' target='_blank'>PubMed</a>"
                }
                for article in filtered_articles
            ])

            # DataFrameを表示
            display(HTML(df.to_html(escape=False, index=False)))

            # エクスポートタブへのナビゲーション
            display(HTML("<p><b>次のステップ:</b> 「エクスポート」タブで絞り込み結果をCSVに保存できます</p>"))

    # 現在の検索結果をエクスポートするボタンのクリックイベント
    def on_export_current_button_clicked(b):
        with export_output:
            clear_output()

            if not searcher.current_articles:
                print("エクスポートするデータがありません。先に検索を実行してください。")
                return

            # ファイル名の取得
            filename = export_filename.value.strip()
            if not filename:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"pubmed_results_{timestamp}.csv"
            elif not filename.endswith('.csv'):
                filename += '.csv'

            # 保存先パスの取得
            path = export_path.value.strip()
            filepath = os.path.join(path, filename) if path else filename

            # エクスポート実行
            saved_path = searcher.export_to_csv(searcher.current_articles, filepath)

            if saved_path:
                print(f"検索結果 ({len(searcher.current_articles)}件) を以下に保存しました:")
                print(f"ファイルパス: {saved_path}")

    # 絞り込み結果をエクスポートするボタンのクリックイベント
    def on_export_filtered_button_clicked(b):
        with export_output:
            clear_output()

            if not searcher.filtered_articles:
                print("エクスポートする絞り込み結果がありません。先に絞り込みを実行してください。")
                return

            # ファイル名の取得
            filename = export_filename.value.strip()
            if not filename:
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"pubmed_filtered_{timestamp}.csv"
            elif not filename.endswith('.csv'):
                filename += '.csv'

            # 保存先パスの取得
            path = export_path.value.strip()
            filepath = os.path.join(path, filename) if path else filename

            # エクスポート実行
            saved_path = searcher.export_to_csv(searcher.filtered_articles, filepath)

            if saved_path:
                print(f"絞り込み結果 ({len(searcher.filtered_articles)}件) を以下に保存しました:")
                print(f"ファイルパス: {saved_path}")

    # 出版年別グラフ表示ボタンのクリックイベント
    def on_visualize_year_button_clicked(b):
        with visualize_output:
            clear_output()

            if not searcher.current_articles:
                print("可視化するデータがありません。先に検索を実行してください。")
                return

            # 現在の検索結果と絞り込み結果の両方を表示
            print("現在の検索結果の出版年分布:")
            searcher.plot_year_distribution(searcher.current_articles)

            if searcher.filtered_articles and searcher.filtered_articles != searcher.current_articles:
                print("\n絞り込み結果の出版年分布:")
                searcher.plot_year_distribution(searcher.filtered_articles)

    # ジャーナル別統計表示ボタンのクリックイベント
    def on_visualize_journal_button_clicked(b):
        with visualize_output:
            clear_output()

            if not searcher.current_articles:
                print("統計を表示するデータがありません。先に検索を実行してください。")
                return

            # ジャーナル統計の取得と表示
            stats = searcher.get_journal_statistics(searcher.current_articles)

            if stats is not None:
                print("ジャーナル別論文数統計:")
                display(stats)

                # 上位10件の円グラフ
                top_journals = stats.head(10)
                plt.figure(figsize=(10, 8))
                plt.pie(top_journals["論文数"], labels=top_journals["ジャーナル名"], autopct='%1.1f%%')
                plt.axis('equal')
                plt.title("論文数上位10ジャーナルの分布")
                plt.show()

                if searcher.filtered_articles and searcher.filtered_articles != searcher.current_articles:
                    print("\n絞り込み結果のジャーナル別統計:")
                    filtered_stats = searcher.get_journal_statistics(searcher.filtered_articles)
                    display(filtered_stats)
            else:
                print("ジャーナル情報がありません。")

    # ボタンにイベントハンドラを登録
    search_button.on_click(on_search_button_clicked)
    filter_button.on_click(on_filter_button_clicked)
    export_current_button.on_click(on_export_current_button_clicked)
    export_filtered_button.on_click(on_export_filtered_button_clicked)
    visualize_year_button.on_click(on_visualize_year_button_clicked)
    visualize_journal_button.on_click(on_visualize_journal_button_clicked)

    # タブの表示
    display(tabs)

# Google Colabでの実行確認と必要なライブラリのインストール
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    # 必要なライブラリのインストール確認
    try:
        import seaborn
    except ImportError:
        print("必要なライブラリをインストールしています...")
        !pip install -q pandas requests ipywidgets matplotlib seaborn tqdm
        print("インストール完了")

# Google Driveのマウント（必要な場合）
def mount_google_drive():
    """Google Driveをマウント"""
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        print("Google Driveをマウントしました。'/content/drive/MyDrive/' 以下にファイルを保存できます。")
    except:
        print("Google Driveのマウントに失敗しました。")

# ツールの説明と使い方
def show_tool_intro():
    """ツールの説明と使い方を表示"""
    display(HTML("""
    <div style="background-color: #f8f9fa; padding: 15px; border-radius: 5px; margin-bottom: 20px;">
        <h1>NCBI/PubMed 高度カスタマイズ論文検索ツール</h1>

        <h3>主な機能:</h3>
        <ul>
            <li>複数の条件による詳細な論文検索</li>
            <li>検索結果を様々な条件でさらに絞り込み</li>
            <li>指定した場所にCSVファイルとして結果を保存</li>
            <li>年別分布やジャーナル統計の可視化</li>
        </ul>

        <h3>使い方:</h3>
        <ol>
            <li><b>検索タブ</b>: キーワード、著者、ジャーナル、年などの条件を指定して検索</li>
            <li><b>絞り込みタブ</b>: 検索結果をさらに詳細な条件で絞り込み</li>
            <li><b>エクスポートタブ</b>: 検索結果または絞り込み結果をCSVに保存（保存先指定可能）</li>
            <li><b>可視化タブ</b>: 検索結果の年別分布やジャーナル統計をグラフで表示</li>
        </ol>

        <h3>Google Driveに保存するには:</h3>
        <p>「エクスポート」タブの保存先に以下のように指定します:</p>
        <code>/content/drive/MyDrive/任意のフォルダ/</code>

        <p><small>※ Google Drive保存を使用する場合は、事前にDriveをマウントしてください</small></p>
    </div>
    """))

# メイン実行関数
def main():
    """メイン実行関数"""
    show_tool_intro()

    # Google Driveマウントボタン
    if IN_COLAB:
        mount_button = widgets.Button(
            description='Google Driveをマウント',
            button_style='info',
            icon='cloud',
            layout=widgets.Layout(width='300px')
        )

        def on_mount_button_clicked(b):
            mount_google_drive()

        mount_button.on_click(on_mount_button_clicked)
        display(mount_button)

    # 検索UIの表示
    create_advanced_search_ui()

# プログラム実行
if __name__ == "__main__" or IN_COLAB:
    main()

Button(button_style='info', description='Google Driveをマウント', icon='cloud', layout=Layout(width='300px'), style…